In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
#jd = pd.read_csv('job_data_dict_30_sample.csv')
#jd = pd.read_csv('job_data_dict_sal_weighted_30_sample.csv')
#jd = pd.read_csv('job_data_dict_all_sal_weighted.csv')
jd = pd.read_csv('job_data_dict_all_sal_weighted_major.csv')

In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9716 entries, 0 to 9715
Data columns (total 12 columns):
Unnamed: 0          9716 non-null int64
major_des           9716 non-null object
jobs                9716 non-null object
jobtitle            9716 non-null object
onetcode            9716 non-null object
major_code          9716 non-null int64
dictionary          9716 non-null object
p_an                9716 non-null float64
p_an_normal         9716 non-null float64
lowct               9716 non-null int64
major_dictionary    9716 non-null object
uid                 9716 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 986.8+ KB


In [4]:
jd.head()

,Unnamed: 0,major_des,jobs,jobtitle,onetcode,major_code,dictionary,p_an,p_an_normal,lowct,major_dictionary,uid
0,2,Management,computer and information systems managers,director of it planning development deployment,11-3021.00,11,director planning development deployment direc...,110000.0,0.097679,0,computer information systems managers computer...,1
1,6,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,45000.0,0.031779,0,financial managers branch department financial...,2
2,16,Management,human resources managers,human resources manager,11-3040.00,11,human resources manager human resources manage...,91197.5,0.078616,0,human resources managers human resources manag...,3
3,28,Management,treasurers and controllers,corporate controller,11-3031.01,11,corporate controller corporate controller year...,115000.0,0.102748,0,treasurers controllers treasurers controllers ...,4
4,31,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,82500.0,0.069798,0,financial managers branch department financial...,5


In [5]:

cat_columns = ['major_des', 'jobs','jobtitle','dictionary','major_dictionary', 'p_an_normal', 'lowct']
major_feat_col = ['major_dictionary', 'p_an_normal']   
feat_col = ['dictionary', 'p_an_normal']
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,major_des,jobs,jobtitle,dictionary,major_dictionary,p_an_normal,lowct
0,Management,computer and information systems managers,director of it planning development deployment,director planning development deployment direc...,computer information systems managers computer...,0.097679,0
1,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.031779,0
2,Management,human resources managers,human resources manager,human resources manager human resources manage...,human resources managers human resources manag...,0.078616,0
3,Management,treasurers and controllers,corporate controller,corporate controller corporate controller year...,treasurers controllers treasurers controllers ...,0.102748,0
4,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.069798,0


In [6]:
# subset the data to the major groups
jd_3 = jd_2[jd_2.major_des=='Computer and Mathematical']
jd_3 = jd_3[jd_3.lowct==0]

In [7]:
jd_3.jobs.value_counts().min()

47

In [8]:
#df_numerical = jd_2.copy()
#encoder_dict = {}

#for col in string_variables:
#    le = preprocessing.LabelEncoder()
#    le.fit(jd_2[col])
#    encoder_dict[col] = le
#    df_numerical[col] = le.transform(jd_2[col])

#df_numerical.head()

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_3[feat_col], jd_3['jobs'], random_state=1)

In [10]:
#feat = list(df_numerical.dictionary.values.flatten())
#type(feat)
#feat = unicode(feat, 'utf-8')

In [11]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
#y_test


In [12]:
X_test.shape

(548L, 2L)

In [13]:
X_train.shape

(1643L, 2L)

In [14]:
type(X_train)

numpy.ndarray

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [16]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train[:,0])
train_dtm

<1643x2239 sparse matrix of type '<type 'numpy.int64'>'
	with 16178 stored elements in Compressed Sparse Row format>

In [17]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test[:,0])
type(test_dtm)

scipy.sparse.csr.csr_matrix

In [18]:
train_features = vect.get_feature_names()
len(train_features)

2239

In [19]:
train_features = train_features + list('s')

In [20]:
len(train_features)

2240

In [21]:
train_sal = np.array(X_train[:,1])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [22]:
test_sal = np.array(X_test[:,1])
test_sal = test_sal[...,None]
type(test_sal)

numpy.ndarray

In [23]:
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [24]:
test_dtm_2 = np.array(test_dtm.todense())

In [25]:
test_dtm_2.shape

(548L, 2239L)

In [26]:
train_dtm_2.shape

(1643L, 2239L)

In [27]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [28]:
type(train_dtm2)
train_dtm2.shape

(1643L, 2240L)

In [29]:
test_dtm2 = np.append(test_dtm_2, test_sal, 1)

In [30]:
# convert train_dtm to a regular array
#train_arr = train_dtm2.toarray()
#train_arr

In [31]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_dtm2, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
546,1346,developer
1817,1108,senior
655,1101,engineer
1872,751,software
73,469,analyst
1497,463,position
1971,451,support
1985,403,systems
1910,390,sr
2185,372,web


In [33]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, y_train)

C:\Anaconda\lib\site-packages\sklearn\naive_bayes.py:427: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  self.feature_count_ += safe_sparse_dot(Y.T, X)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm2)
preds

array(['computer support specialists',
       'information technology project managers', 'web developers',
       'web developers', 'web developers',
       'computer software engineers applications', 'web developers',
       'web developers', 'computer programmers',
       'computer support specialists', 'computer support specialists',
       'computer software engineers applications', 'web developers',
       'computer software engineers applications',
       'software quality assurance engineers and testers',
       'network and computer systems administrators',
       'computer software engineers applications',
       'computer support specialists',
       'computer software engineers applications',
       'software quality assurance engineers and testers',
       'computer software engineers applications',
       'computer software engineers applications',
       'computer systems engineers architects',
       'network and computer systems administrators',
       'information tech

In [35]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.806569343066
[[  4   0   2   0   0   1   0   0   0   0   0   7]
 [  0   7   1   0   0   2   1   0   1   5   0   0]
 [  1   0  94   0   0   0   4   0   2   1   0  13]
 [  0   0   6   2   0   0   0   0   0   3   0   0]
 [  0   1   0   0  56   2   1   0   1   2   0   0]
 [  0   0   3   0   1  20   0   0   2   0   1   1]
 [  0   0   1   0   0   0  11   0   0   3   0   0]
 [  0   1   2   0   0   1   0  11   0   1   0   1]
 [  0   0   0   0   0   0   0   0  30   0   0   1]
 [  0   0   5   0   3   0   0   0   0  73   0   0]
 [  0   0   2   0   0   1   0   0   0   1  15   0]
 [  0   0  16   0   0   0   1   0   1   0   0 119]]


In [36]:
# predict (poorly calibrated) probabilities and calculate AUC
#probs = nb.predict_proba(test_dtm2)[:, 1]
#probs
#print(metrics.roc_auc_score(y_test, probs))

In [37]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [38]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [39]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

computer programmers       0.80      0.29      0.42        14
computer security specialists       0.78      0.41      0.54        17
computer software engineers applications       0.71      0.82      0.76       115
computer software engineers systems software       1.00      0.18      0.31        11
computer support specialists       0.93      0.89      0.91        63
computer systems analysts       0.74      0.71      0.73        28
computer systems engineers architects       0.61      0.73      0.67        15
database administrators       1.00      0.65      0.79        17
information technology project managers       0.81      0.97      0.88        31
network and computer systems administrators       0.82      0.90      0.86        81
software quality assurance engineers and testers       0.94      0.79      0.86        19
web developers       0.84      0.87      0.85       137

avg / total       0.82      0.81      0.80       54

In [32]:
# use SVM
from sklearn import svm
from sklearn import metrics
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm2, y_train)
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm2)

print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.795620437956
[[  8   0   1   0   0   0   0   0   0   0   0   5]
 [  0  13   0   0   0   2   0   0   1   1   0   0]
 [  1   2  89   0   0   0   9   0   1   1   0  12]
 [  0   0   3   5   0   0   0   0   0   3   0   0]
 [  0   0   0   0  56   1   1   0   2   3   0   0]
 [  3   0   3   0   3  17   0   0   1   0   1   0]
 [  0   0   0   0   0   1  11   0   0   3   0   0]
 [  0   1   2   0   0   1   0  12   0   1   0   0]
 [  0   0   0   0   1   1   0   0  29   0   0   0]
 [  0   1   3   2   3   2   1   0   1  68   0   0]
 [  0   0   2   0   1   0   0   0   0   1  15   0]
 [  4   0  16   0   1   0   1   0   2   0   0 113]]


In [41]:
#from sklearn.cross_validation import cross_val_score
#cross_val_score(nb, train_dtm, y_train, cv=5, scoring='accuracy').mean()

In [42]:
# try using all of the data

In [43]:
train_dtm = vect.fit_transform(jd_3['dictionary'])
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [44]:
train_sal = np.array(jd_3['p_an_normal'])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [45]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [46]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, jd_3['jobs'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [47]:
preds = nb.predict(train_dtm2)
preds.shape

(2191L,)

In [48]:

print(classification_report(jd_3['jobs'], preds) )

             precision    recall  f1-score   support

computer programmers       1.00      0.49      0.66        80
computer security specialists       0.98      0.75      0.85        61
computer software engineers applications       0.83      0.89      0.86       484
computer software engineers systems software       0.97      0.68      0.80        47
computer support specialists       0.95      0.92      0.94       247
computer systems analysts       0.85      0.86      0.85       122
computer systems engineers architects       0.78      0.76      0.77        74
database administrators       0.94      0.83      0.88        58
information technology project managers       0.89      0.96      0.92       113
network and computer systems administrators       0.90      0.94      0.92       316
software quality assurance engineers and testers       0.95      0.92      0.93        84
web developers       0.88      0.93      0.90       505

avg / total       0.89      0.88      0.88      219

In [49]:

target_predicted = nb.predict(train_dtm2)
target_prob = nb.predict_proba(train_dtm2)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_3['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,computer programmers,computer security specialists,computer software engineers applications,computer software engineers systems software,computer support specialists,computer systems analysts,computer systems engineers architects,database administrators,information technology project managers,network and computer systems administrators,software quality assurance engineers and testers,web developers,All
True,,,,,,,,,,,,,
computer programmers,39,0,12,0,0,5,0,0,0,0,0,24,80
computer security specialists,0,46,2,0,0,2,1,0,0,10,0,0,61
computer software engineers applications,0,0,433,0,2,1,5,1,1,5,0,36,484
computer software engineers systems software,0,0,11,32,0,0,0,0,0,4,0,0,47
computer support specialists,0,0,2,0,228,7,2,1,2,3,1,1,247
computer systems analysts,0,0,7,0,1,105,2,0,4,1,1,1,122
computer systems engineers architects,0,0,8,1,0,0,56,0,2,7,0,0,74
database administrators,0,1,5,0,0,1,1,48,0,1,0,1,58
information technology project managers,0,0,1,0,1,0,2,0,108,1,0,0,113


In [50]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, jd_3['jobs'], cv=5, scoring='accuracy').mean()

0.78327287700029768